# Neural Networks

Imports and Helper Functions
---

In [21]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display
from sklearn import datasets as skdatasets
from ipywidgets import interactive_output

import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.nn.functional as F
import torch.optim as optim

%matplotlib inline

try:
    import demo_datasets as ds
except:
    !curl https://raw.githubusercontent.com/reml-lab/MLDemos/refs/heads/main/demo_datasets.py -o demo_datasets.py
    import demo_datasets as ds

## Model Creation

In [22]:
#Define a basic MLP with pyTorch
class MLP(nn.Module):
    def __init__(self, K):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(2, K)
        self.out = nn.Linear(K, 1)
        self.ceriation = nn.BCELoss()

    def _predict_proba(self, X):
        h1 = torch.sigmoid(self.fc1(X))
        yhat = torch.sigmoid(self.out(h1))
        return yhat

    def forward(self, X, y):
        yhat = self._predict_proba(X)
        loss = self.ceriation(yhat, y)
        return loss

    def predict_proba(self, X):
        X = Variable(torch.from_numpy(X).float())
        yhat = self._predict_proba(X)
        return yhat.data.numpy()

    def fit(self, X, y, verbose=False):
        X2 = torch.from_numpy(X).float()
        y2 = torch.from_numpy(y[:, np.newaxis]).float()
        optimizer = optim.Adam(self.parameters(), lr=0.1, weight_decay=1e-10)

        old_loss = np.inf
        for epoch in range(100):
            optimizer.zero_grad()
            loss = self(Variable(X2), Variable(y2))
            loss.backward()
            optimizer.step()
            if np.abs(loss.data.item() - old_loss) / np.abs(loss.data.item()) < 1e-6:
                break
            old_loss = loss.data.item()
            if verbose:
                print('==>>> epoch: {}, train loss: {:.6f}'.format(epoch, loss.data.item()))
        if verbose:
            print('==>>> epoch: {}, train loss: {:.6f}'.format(epoch, loss.data.item()))

## Dataset Generation

In [23]:
datasets={}
def make_datasets():
    global datasets
    datasets["Blobs"]={"train":ds.make_blobs(),"test":ds.make_blobs()}
    datasets["XOR"]={"train":ds.make_xor(),"test":ds.make_xor()}
    datasets["Circles"]={"train":ds.make_circles(),"test":ds.make_circles()}
    datasets["Moons"]={"train":ds.make_moons(),"test":ds.make_moons()}
    datasets["Random"]={"train":ds.make_random(),"test":ds.make_random()}

make_datasets()

## Plotting Function

In [24]:
def plot_nn_classifier(dataset='Dataset 1', N=10, K=5,t=0, showtrain=True,showtest=False):
    plt.figure(figsize=(10, 6))
    plt.rcParams.update({'font.size': 12})

    if dataset is not None and N > 0:
        # Gather dataset

        Xtrain, Ytrain = datasets[dataset]["train"]
        Xtrain = Xtrain[:N]
        Ytrain = Ytrain[:N]

        Xtest, Ytest = datasets[dataset]["test"]

        # Plot original data

        if(showtest):
            ind1 = (Ytest == 1)
            indm1 = (Ytest == 0)
            plt.plot(Xtest[indm1, 0], Xtest[indm1, 1], 'rx', alpha=0.5,markersize=5)
            plt.plot(Xtest[ind1, 0], Xtest[ind1, 1], 'bx', alpha=0.5, markersize=5) 

        if(showtrain):
            ind1 = (Ytrain == 1)
            indm1 = (Ytrain == 0)
            plt.plot(Xtrain[indm1, 0], Xtrain[indm1, 1], 'rs', alpha=0.5)
            plt.plot(Xtrain[ind1, 0], Xtrain[ind1, 1], 'bs', alpha=0.5)   

        # Create the classifier
        model = MLP(K)
        model.fit(Xtrain, Ytrain)

        # Calculate prediction error

        Yprob = np.reshape(model.predict_proba(Xtrain), -1)
        Yhat = 1 * (Yprob >= 0.5)
        Rtrain = np.mean(Yhat != Ytrain)

        Yprob = np.reshape(model.predict_proba(Xtest), -1)
        Yhat = 1 * (Yprob >= 0.5)
        Rtest = np.mean(Yhat != Ytest)

        # Create contours for background plot

        gx1, gx2 = np.meshgrid(np.arange(-10, 10, 1 / 10.0), np.arange(-10, 10, 1 / 10.0))
        gx1l = gx1.flatten()
        gx2l = gx2.flatten()
        gx = np.vstack((gx1l, gx2l)).T
        gyhat = model.predict_proba(gx)[:, 0]
        gyhat = gyhat.reshape(gx1.shape)

        if (not all(gyhat.flatten() == 0)):
            contours = plt.contour(gx1, gx2, gyhat, levels=[0.5], linewidths=2, colors=["k"])
            plt.imshow(gyhat, interpolation='nearest', extent=(-10, 10, -10, 10), aspect='auto', origin='lower',
                       cmap=plt.cm.bwr_r, alpha=0.50)
            plt.clim(0, 1)
            plt.colorbar()

        plt.xlim(-10, 10)
        plt.ylim(-10, 10)
        plt.title(f"MLP (K={K}, Train Error Rate: {Rtrain:.4g}, Test Error Rate: {Rtest:.4g})")
        plt.xlabel("X0")
        plt.ylabel("X1")
        plt.grid(True)
        plt.legend(["Class 0", "Class 1"], loc='upper left')
        plt.show()

In [25]:
def regenerate_data(b):
    make_datasets()
    wtrigger.value = wtrigger.value + 1

def reinit_model(b):
    wtrigger.value = wtrigger.value + 1

wd = widgets.Dropdown(
    options=datasets.keys(),
    description="Dataset"
)
wn = widgets.IntSlider(value=10, min=1, max=1000, step=1, description="N", continuous_update=False)
wk = widgets.IntSlider(
    value=1,
    min=1,
    max=50,
    step=1,
    description='K:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True
)
wrandomize = widgets.Button(description="Re-Generate Data")
wtrigger = widgets.IntText(value=0, layout=widgets.Layout(display="none"))
wtrain = widgets.Checkbox(value=True, description='Show Train', disabled=False)
wtest = widgets.Checkbox(value=False, description='Show Test', disabled=False)
wrandomize.on_click(regenerate_data)
winit = widgets.Button(description="Re-Init Model")

wrandomize.on_click(regenerate_data)
winit.on_click(reinit_model)

out = interactive_output(
    plot_nn_classifier,
    {
        "dataset": wd,
        "N": wn,
        "K": wk,
        "t":wtrigger,
        "showtrain":wtrain,
        "showtest":wtest
    }
)

#out.layout.height = '400px'
box1 = widgets.HBox([wd, wrandomize, wtrain,wtest ])
box2 = widgets.HBox([ wn, wk,winit])
ui = widgets.VBox([box1,box2]);

display(ui)
display(out)

Output()